# Tree

In [54]:
import pandas as pd
import numpy as np
import math

## Decision Trees

https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea

In [70]:
class DecisionTreeEntropy():
    """
    """
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        
        
    def find_best_split(self, ind_var, dep_var):
        """
        Find best split
        
        ind_var: the column we split
        dep_var: target variable (independent variable)
        """
        min_entropy = 10
        n = len(dep_var)
        
        for value in list(map(np.unique, ind_var)):  # iterate through each value in the column
            y_predict = ind_var < value  # seperate y into two groups
            entropy = self.get_entropy(y_predict, dep_var)
            
            if entropy <= min_entropy:
                min_entropy = entropy
                cutoff = value
            
        return min_entropy, cutoff
    
    
    def find_best_split_all(self, x, y):
        """
        Find best split of all feature
        """
        col = None
        min_entropy = 1
        cutoff = None
        
        for i, c in enumerate(x.T):
            entropy, feature_cutoff = self.find_best_split(c, y) # find best split of feature
            
            if entropy == 0:
                return i, feature_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = feature_cutoff
                
        return col, cutoff, min_entropy
    
    
    def fit(self, x, y, par_node = {}, depth = 0):
        """
        x: Feature set
        y: target variable
        par_node: will be the tree generated for this x and y. 
        depth: the depth of the current layer
        """
        if par_node is None: return None
        elif len(y) == 0: return None
        elif all(x == y[0] for x in y): return {'val': y[0]}
        elif depth >= self.max_depth: return None
        else:
            col, cutoff, entropy = self.find_best_split_all(x, y)
            
            y_left = y[x[:, col] < cutoff]  # left hand side
            y_right = y[x[:, col] >= cutoff]  # right hand side
        
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                    'cutoff':cutoff, 'val': np.round(np.mean(y))}  # save the information 
            
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)  # tree for the left hand side data 
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)  # tree for right hand side trees
            
            self.depth += 1   # increase the depth since we call fit once
            self.trees = par_node  
            return par_node
        
    def predict(self, x):
        """
        """
        pass
        
    ############### ENTROPY ###############
    def entropy(self, count, total):
        """
        Formula for entropy
        """
        prob = count / total
        return -prob * math.log(prob, 2)
    
    
    def calc_entropy(self, c1, c2):
        """
        Calculate entropy of group date
        
        c1: count of one class
        c2: count of another class
        """
        if c1 == 0 or c2 == 0:  # entropy 0 when there is only one class
            return 0
        
        return self.entropy(c1, c1 + c2) + self.entropy(c2, c1 + c2)
    
    
    def entropy_one_division(self, division):
        """
        """
        s = 0
        n = len(division)
        classes = set(division)
        
        for c in classes:
            n_c = sum(division == c)
            entropy = (n_c/n) * self.calc_entropy(sum(division == c), sum(division != c))
            s += entropy

            
        return s, n
    
    
    def get_entropy(self, y_train, y_true):
        """
        """
        if len(y_train) != len(y_true):
            raise Exception('Length of y_train and y_true is not same')
            
        n = len(y_true)
        
        # get left hand side entropy
        s_left, n_left = self.entropy_one_division(y_true[y_train])  # get # of True
        
        # get right hand side entropy
        s_right, n_right = self.entropy_one_division(y_true[~y_train])  # get # of False
        
        return (n_left/n)*s_left + (n_right/n)*s_right
        

In [71]:
from sklearn.datasets import load_iris
from pprint import pprint

iris = load_iris()

x = iris.data
y = iris.target
clf = DecisionTreeEntropy(max_depth=3).fit(x, y)
pprint(clf)

{'col': 'petal width (cm)',
 'cutoff': array([1.]),
 'index_col': 3,
 'left': {'val': 0},
 'right': {'col': 'petal width (cm)',
           'cutoff': array([1.8]),
           'index_col': 3,
           'left': {'col': 'petal length (cm)',
                    'cutoff': array([5.]),
                    'index_col': 2,
                    'left': None,
                    'right': None,
                    'val': 1.0},
           'right': {'col': 'petal length (cm)',
                     'cutoff': array([4.9]),
                     'index_col': 2,
                     'left': None,
                     'right': {'val': 2},
                     'val': 2.0},
           'val': 2.0},
 'val': 1.0}
